## Problem Definition
In this simple ANN implementation, we use Artificial Neural Network and want to predict that the customer leave the bank or not according to the features in the dataset(csv file).

In [ ]:
# Unzip the uploaded data into Google Drive
# !unzip "drive/MyDrive/Datasets/ML-AZ-Datasets/2.1 Machine Learning A-Z (Codes and Datasets).zip" -d "drive/MyDrive/Datasets/ML-AZ-Datasets"

## Importing the tools

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.7.0'

## Step-1: Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv("drive/MyDrive/Datasets/ML-AZ-Datasets/Churn_Modelling.csv")
# X = df.drop("Exited", axis=1)
# y = df["Exited"]
X = dataset.iloc[:, 3: -1].values
y = dataset.iloc[:, -1].values

In [ ]:
len(dataset)

10000

In [ ]:
X.shape, y.shape

((10000, 10), (10000,))

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


### Checking data is missing or not

In [ ]:
dataset.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Encoding categorical data
Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
X[:, 2] = LabelEncoder().fit_transform(X[:, 2]) # Just column 2 with all rows

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column_transform = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(column_transform.fit_transform(X))  # np.array is used to prevent error "cant convert tensor" message if we predict on X_test

In [ ]:
print(X)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=37)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

## Part-2: Building the ANN model

### Initializing the ANN

In [ ]:
model = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [ ]:
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [ ]:
model.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [ ]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part-3: Training the ANN

### Compiling the model

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN to the Training set

In [ ]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5652 - accuracy: 0.7796
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4738 - accuracy: 0.7954
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4413 - accuracy: 0.7971
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.8040
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4043 - accuracy: 0.8284
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3902 - accuracy: 0.8367
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3783 - accuracy: 0.8396
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3688 - accuracy: 0.8425
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3613 - accuracy: 0.8487
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3556 - accura

## Part-4: Making the predictions and evaluating the model

First use our model to predict on single data of the customer with the following features. Let's predict 
the customer will leave(1) the bank or not(0).

* Geography: France
* Credict Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: $60000
* Number of Product: 2
* Does this customer have a credit card? Yes

* Is the customer an Active Member? Yes
* Estimated Salary: $50000


In [ ]:
cus_predict = model.predict(standard_scaler.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
print(cus_predict > 0.5)

[[False]]


As it is predict `False-0`, our ANN model predicts the customer with that infomations(features) will not leave the bank and stay the bank!

**Importance note 1:** Notice that the values of features were all input in a double pair of square brackets. That's because the `predict` method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets make the input exactly a 2D array.

**Importance note 2:** Notice also that the "France" country was not input as a string in the last column but but "1, 0, 0" in the fist three column. That's because the `predict` method expect one-hot-encoded values of the state, and as we see in first row of the matrix of feature X, "France" was encoded as "1, 0, 0". 

### Predicting the Test set results

In [ ]:
X_test

array([[ 1.00702467e+00, -5.81584228e-01, -5.78504971e-01, ...,
        -1.55619021e+00, -1.03901035e+00, -1.74421562e-05],
       [-9.93024329e-01,  1.71944140e+00, -5.78504971e-01, ...,
         6.42594969e-01, -1.03901035e+00, -1.24162142e+00],
       [-9.93024329e-01,  1.71944140e+00, -5.78504971e-01, ...,
        -1.55619021e+00,  9.62454323e-01,  1.38166993e-01],
       ...,
       [-9.93024329e-01,  1.71944140e+00, -5.78504971e-01, ...,
        -1.55619021e+00, -1.03901035e+00, -6.19326601e-01],
       [ 1.00702467e+00, -5.81584228e-01, -5.78504971e-01, ...,
         6.42594969e-01, -1.03901035e+00, -4.52924855e-01],
       [-9.93024329e-01,  1.71944140e+00, -5.78504971e-01, ...,
         6.42594969e-01, -1.03901035e+00,  1.19590255e+00]])

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 0]
 [1 1]
 ...
 [1 1]
 [0 0]
 [0 0]]


### Making Confusion Matrix

Sklearn Confusion Matrix Design:
Predicted Label is on horizontal(X-axis) &
True Label is on vertical(Y-axis)

`0`

`1`

      0    1
* Top Left - (0, 0) > TN
* Right Bottom - (1, 1) > TP
* Left Bottom - (0, 1) > FN
* Top Right - (1, 0) - FP

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[1534,   66],
       [ 224,  176]])

In [ ]:
accuracy_score(y_test, y_pred)

0.855